In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [2]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [3]:
design.chips['C_chip'].size['size_x']='3mm'
design.chips['C_chip'].size['size_y']='3mm'
design.chips['Q_chip'].size['size_x']='2mm'
design.chips['Q_chip'].size['size_y']='2mm'
design.chips['Q_chip'].size['center_z']='6um'

In [4]:
design.get_chip_z('Q_chip')

'6um'

In [5]:

cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-300um',
                                            pos_y='-500um',
                                            chip='Q_chip',
                                            pad_width='370um',
                                            pad_height='100um',
                                            pad_gap='80um',
                                            pocket_width='680um',
                                            pocket_height='680um'))]
                                            
qubits=q0
q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = StraightUpCoupler(design, "c0", options=dict(pos_x=f"{q0x-120}um",
                                              pos_y=f"{q0y+90}um",
                                              pad_width="110um",
                                              pad_height="100um",
                                              cpw_stub_height="200um",
                                              chip="C_chip"))
design.rebuild()
gui.rebuild()
gui.autoscale()

In [6]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-700um',
                                         pos_y = '800um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '700um',
                                         pos_y = '800um',
                                         orientation='180',
                                         lead_length = '30um'))                                        

In [7]:
stg=ShortToGround(design, 'stg', options=Dict(chip="C_chip",
                                                pos_x='0.65mm',
                                                pos_y='-0.2mm',
                                                orientation='270'))

In [8]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [9]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

# Resonator length
3.85mm

In [10]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [11]:
resonator_length_4=[]
a=find_resonator_length(7.5,10,6,4)
a=a.strip('mm')
a=float(a)-0.25 +0.5-0.23-0.5# subtract coupler length, for +0.5Ghz, coupler length
resonator_length_4.append(a)
print(resonator_length_4)


[3.5775622473488387]


In [12]:
jogs1=OrderedDict()
jogs1[0]=['R','300um']
jogs1[1]=['R', '300um']
jogs1[2]=['L','700um']
jogs1[3]=['R','200um']
jogs1[4]=['R','670um']
jogs1[5]=['L','200um']
jogs1[6]=['L','740um']


In [13]:
rr0=connect('rr0', 'c0', 'in','stg', 'short',
            resonator_length_4[0],jogs1,jogs, 
            '100um','0um', '944um', '100um')
            

# EPR Analysis

In [36]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [37]:
hfss.start()

INFO 03:37PM [connect_project]: Connecting to Ansys Desktop API...
INFO 03:37PM [load_ansys_project]: 	Opened Ansys App
INFO 03:37PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 03:37PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Flipchip Bump
INFO 03:37PM [connect_design]: 	Opened active design
	Design:    flip7um_V6 [Solution type: Eigenmode]
INFO 03:37PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 03:37PM [connect]: 	Connected to project "Flipchip Bump" and design "flip7um_V6" 😀 



True

In [38]:
hfss.activate_ansys_design("flip6um_V6", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','stg','rr0','fl0'],[],
                    box_plus_buffer=False)

03:37PM 52s WARNING [activate_ansys_design]: The design_name=flip6um_V6 was not in active project.  Designs in active project are: 
['flip7um_V6'].  A new design will be added to the project.  
INFO 03:37PM [connect_design]: 	Opened active design
	Design:    flip6um_V6 [Solution type: Eigenmode]
WARNING 03:37PM [connect_setup]: 	No design setup detected.
WARNING 03:37PM [connect_setup]: 	Creating eigenmode default setup.
INFO 03:37PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [39]:
hfss.close()


Warning! 3 COM references still alive
Ansys will likely refuse to shut down


True

# LOM Analysis

In [14]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [15]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='5.69',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '5.69',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [19]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 02:54PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpwxoi_seg.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5690000000, Maxwell, 1, False
INFO 02:54PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp7ha558nj.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5690000000, Maxwell, 1, False
INFO 02:54PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp34g8wjpd.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5690000000, Maxwell, 2, False
INFO 02:54PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmppxjepuoh.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5690000000, Maxwell, 3, False
INFO 02:54PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpm08mduoz.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5690000000, Maxwell, 4, False
INFO 02:54

,ground,pad_bot_Q0,pad_top_Q0,quad_coupler_c0
ground,375.520,-127.080000,-107.480,-75.318000
pad_bot_Q0,-127.080,139.690000,-11.412,-0.034146
pad_top_Q0,-107.480,-11.412000,139.180,-19.259000
quad_coupler_c0,-75.318,-0.034146,-19.259,94.685000


In [20]:
c1.setup.junctions = Dict({'Lj': 12, 'Cj': 2})
c1.setup.freq_readout = 7.36
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 4.974107 [GHz]
EC 251.969962 [MHz]
EJ 13.616300 [GHz]
alpha -287.493205 [MHz]
dispersion 4.981430 [KHz]
Lq 11.995161 [nH]
Cq 76.875148 [fF]
T1 106.262556 [us]

**Coupling Properties**

tCqbus1 -9.626683 [fF]
gbus1_in_MHz -133.442592 [MHz]
χ_bus1 -1.674127 [MHz]
1/T1bus1 1497.751877 [Hz]
T1bus1 106.262556 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,5.639131,329.226044,13.6163,-385.014394,61.270724,[-127.68842823194481],[-3.540965372315225],3.540965,127.688428
2,5.093355,264.962298,13.6163,-303.598109,8.205295,[-152.77845903754869],[-2.526408295211723],2.526408,152.778459
3,4.945323,248.888972,13.6163,-283.69142,4.399916,[-140.08559581887533],[-1.784112961922001],1.784113,140.085596
4,4.969508,251.476223,13.6163,-286.883509,4.884078,[-133.94095090441323],[-1.6776211386619857],1.677621,133.940951
5,4.966147,251.115742,13.6163,-286.438477,4.814024,[-133.84380437969276],[-1.668631762552019],1.668632,133.843804
6,4.974107,251.969962,13.6163,-287.493205,4.98143,[-133.44259163638938],[-1.674126658078233],1.674127,133.442592


In [21]:
c1.sim.close()

Warning! 3 COM references still alive
Ansys will likely refuse to shut down
